# Introduction to MPI on Amazon SageMaker

Message Passing Interface (MPI) is the fundamental communication protocol for programming parallel computer programs. See its [wiki page](https://en.wikipedia.org/wiki/Message_Passing_Interface). [Open MPI](https://www.open-mpi.org/projects/user-docs/) is the implementation that's used as a basic building block for distributed training systems. 

In Python programs, you can interact with Open MPI APIs via [mpi4py](https://mpi4py.readthedocs.io/en/stable/overview.html) and easily convert your single-process python program into a parallel python program. 

Parallel processes can exist on one host (e.g. one EC2 instance) or multiple hosts (e.g. many EC2 instances). It's trivial to set up a parallel cluster (comm world, in MPI parlance) on one host via Open MPI, but it is less straight-forward to set up an MPI comm world across multiple instances. 

SageMaker does it for you. In this tutorial, you will go through a few basic (but exceeding important) [MPI communications](https://mpi4py.readthedocs.io/en/stable/tutorial.html) on SageMaker with **multiple instances** and you will verify that parallel processes across instances are indeed talking to each other. Those basic communications are the fundamental building blocks for distributed training.

## Environment 
We assume Open MPI and mpi4py have been installed in your environment. This is the case for SageMaker Notebook Instance or Studio. 

## Inspect the Python Program

In [ ]:
!pygmentize mpi_demo.py

See the program in action with 2 parallel processes on your current environment. Make sure you have at least 2 cores.

In [ ]:
!mpirun -np 2 python mpi_demo.py

## Scale it on SageMaker
You can run the above program with $n$ processes per host across $N$ hosts on SageMaker (and get a comm world of size $n\times N$). In the remaining of this notebook, you will use SageMaker TensorFlow deep learning container to run the above program. There is no particular reason for the choice, all SageMaker deep learning containers have Open MPI installed. So feel free to replace it with your favorite DLC. 

Check out the [SageMaker Python SDK Docs](https://sagemaker.readthedocs.io/en/stable/api/training/smd_model_parallel_general.html?highlight=mpi%20paramters#mpi-parameters) for the parameters needed to set up a distributed training job with MPI. 

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow
from sagemaker.pytorch import PyTorch

#role = get_execution_role()
role = 'arn:aws:iam::976939723775:role/service-role/AmazonSageMaker-ExecutionRole-20210317T133000'
# Running 2 processes per host
# if we use 3 instances,
# then we should see 6 MPI processes

distribution = {"mpi": {"enabled": True, "processes_per_host": 2}}

tfest = TensorFlow(
    entry_point="mpi_demo.py",
    role=role,
    framework_version="2.3.0",
    distribution=distribution,
    py_version="py37",
    instance_count=3,
    instance_type="ml.c5.2xlarge",  # 8 cores
    output_path="s3://" + sagemaker.Session().default_bucket() + "/" + "tf_mpi",
)

torchfest = PyTorch(
    entry_point="mpi_demo.py",
    role=role,
    framework_version="1.10.0",
    distribution=distribution,
    py_version="py38",
    instance_count=3,
    instance_type="ml.c5.2xlarge",  # 8 cores
    output_path="s3://" + sagemaker.Session().default_bucket() + "/" + "torch_mpi",
)

In [ ]:
tfest.fit(wait=False)

In [ ]:
torchfest.fit(wait=True)

The stdout "Number of MPI processes that will talk to each other:  6" indicates that the processes on all hosts are included in the comm world. 

## Conclusion
In this notebook, you went through some fundamental MPI operations, which are the bare bones of inner workings of many distributed training frameworks. You did that on SageMaker with multiple instances. You can scale up this set up to include more instances in a real ML project.